In [57]:
import pandas as pd


In [58]:
data=pd.read_csv(r"PS_20174392719_1491204439457_log.csv")
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [21]:
pd.crosstab(data.type,df.isFraud)

isFraud,0,1
type,,
CASH_IN,1399284,0
CASH_OUT,2233384,4116
DEBIT,41432,0
PAYMENT,2151495,0
TRANSFER,528812,4097


In [59]:
fraudData=data[(data.type=="CASH_OUT")|(data.type=="TRANSFER")]

In [60]:
fraudData.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.0,0.0,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.00,1,0
15,1,CASH_OUT,229133.94,C905080434,15325.0,0.0,C476402209,5083.0,51513.44,0,0
19,1,TRANSFER,215310.30,C1670993182,705.0,0.0,C1100439041,22425.0,0.00,0,0
24,1,TRANSFER,311685.89,C1984094095,10835.0,0.0,C932583850,6267.0,2719172.89,0,0


In [33]:
fraudData.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [61]:
x=fraudData[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest','type']]
y=fraudData.isFraud

In [63]:
x_dummy=pd.get_dummies(data=x,columns=['type'],drop_first=True)

In [64]:
x_dummy.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_TRANSFER
2,181.00,181.0,0.0,0.0,0.00,1
3,181.00,181.0,0.0,21182.0,0.00,0
15,229133.94,15325.0,0.0,5083.0,51513.44,0
19,215310.30,705.0,0.0,22425.0,0.00,1
24,311685.89,10835.0,0.0,6267.0,2719172.89,1


In [65]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [66]:

x_train,x_test,y_train,y_test=train_test_split(x_dummy,y,random_state=10)
std=StandardScaler()
std.fit(x_train)
x_train_std=std.transform(x_train)
x_test_std=std.transform(x_test)

In [70]:
#deciscion tree
model_dt=DecisionTreeClassifier()
model_dt.fit(x_train_std,y_train)
print("Train Score: ",model_dt.score(x_train_std,y_train))
print("Test Score: ",model_dt.score(x_test_std,y_test))
print("*******************************************")
print(pd.crosstab(y_test,model_dt.predict(x_test_std)))

Train Score:  1.0
Test Score:  0.9993127376000392
col_0         0     1
isFraud              
0        690350   219
1           257  1777


In [71]:
#Random Forest
model_Rf=RandomForestClassifier()
model_Rf.fit(x_train,y_train)
print('Train Score: ',model_Rf.score(x_train,y_train))
print('Train Score: ',model_Rf.score(x_test,y_test))
print("*******************************************")
print(pd.crosstab(y_test,model_Rf.predict(x_test_std)))

Train Score:  1.0
Train Score:  0.9992203325714731
*******************************************


NameError: name 'model_dt' is not defined

In [74]:
pd.crosstab(y_test,model_Rf.predict(x_test_std))

col_0,0,1
isFraud,,
0,18466,672103
1,114,1920


In [78]:
yhat_train=model.predict(x_train_std)
yhat_test=model.predict(x_test_std)

In [79]:
from sklearn.metrics import precision_score,recall_score,f1_score

In [81]:
print("precision:",precision_score(y_test,yhat_test))
print("Recall:",recall_score(y_test,yhat_test))
print("F1:",f1_score(y_test,yhat_test))

precision: 0.8902805611222445
Recall: 0.8736479842674533
F1: 0.8818858560794045


In [62]:
#feature importancs-tells us about important feature in predicting .only in random forest 
#recall

model.feature_importances_

array([0., 0., 0., 0., 0., 0.])

In [ ]:
#Logistic regression
model_log=LogisticRegression()
model_log.fit(x_train_std,y_train)
print("Train Score: ",model_log.score(x_train_std,y_train))
print("Test Score: ",model_log.score(x_test_std,y_test))
print("*******************************************")
print(pd.crosstab(y_test,model_dt.predict(x_test_std)))

In [1]:
pd.DataFrame({'Feature':x_train.columns,'Importance':model_Rf.feature_importances_}).sort_values("Importance",ascending=False)

NameError: name 'pd' is not defined